<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [1]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: C:\Users\Nikola\PycharmProjects\VNP\lab3\ElectricCarData.csv

  0%|          | 0.00/8.20k [00:00<?, ?B/s]
100%|##########| 8.20k/8.20k [00:00<00:00, 8.18MB/s]


### Import the required libraries

In [1]:
import pandas as pd

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [2]:
data = pd.read_csv('ElectricCarData.csv')

In [3]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    object 
 1   Model            103 non-null    object 
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    object 
 7   RapidCharge      103 non-null    object 
 8   PowerTrain       103 non-null    object 
 9   PlugType         103 non-null    object 
 10  BodyStyle        103 non-null    object 
 11  Segment          103 non-null    object 
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 11.4+ KB


### Encode string variables

In [5]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

In [6]:
data.Brand.value_counts()

Brand
Tesla          13
Audi            9
Volkswagen      8
Nissan          8
Skoda           6
Porsche         5
Kia             5
Renault         5
BMW             4
Ford            4
Opel            3
Hyundai         3
Smart           3
Byton           3
Mercedes        3
Honda           2
Peugeot         2
Fiat            2
Volvo           1
MG              1
Lucid           1
Polestar        1
Mini            1
Mazda           1
Lexus           1
Aiways          1
Lightyear       1
SEAT            1
CUPRA           1
Jaguar          1
Citroen         1
DS              1
Sono            1
Name: count, dtype: int64

In [7]:
encoder = LabelEncoder()
data['Brand'] = encoder.fit_transform(data['Brand'])

In [8]:
data.Model.value_counts()

Model
e-Soul 64 kWh                      2
Model 3 Long Range Dual Motor      1
2                                  1
iX3                                1
e                                  1
                                  ..
Ariya 63kWh                        1
e-tron S Sportback 55 quattro      1
Ariya e-4ORCE 63kWh                1
Ariya e-4ORCE 87kWh Performance    1
M-Byte 95 kWh 2WD                  1
Name: count, Length: 102, dtype: int64

In [9]:
encoder = LabelEncoder()
data['Model'] = encoder.fit_transform(data['Model'])

In [10]:
data.FastCharge_KmH.value_counts()

FastCharge_KmH
230    6
540    5
440    5
380    5
-      5
560    4
190    4
470    4
170    3
260    3
210    3
420    3
220    2
590    2
710    2
490    2
270    2
650    2
620    2
550    2
340    2
480    2
330    2
390    2
350    2
450    2
940    1
850    1
930    1
780    1
250    1
610    1
570    1
240    1
910    1
730    1
920    1
900    1
520    1
410    1
770    1
430    1
890    1
360    1
460    1
810    1
290    1
740    1
510    1
320    1
500    1
Name: count, dtype: int64

In [11]:
data['FastCharge_KmH'] = data['FastCharge_KmH'].apply(lambda x: 0 if x == '-' else x)
data['FastCharge_KmH'] = data['FastCharge_KmH'].astype(int)

In [12]:
encoder = LabelEncoder()
data['RapidCharge'] = encoder.fit_transform(data['RapidCharge'])

In [13]:
data.PowerTrain.value_counts()

PowerTrain
AWD    41
FWD    37
RWD    25
Name: count, dtype: int64

In [14]:
encoder = LabelEncoder()
data['PowerTrain'] = encoder.fit_transform(data['PowerTrain'])

In [15]:
data.PlugType.value_counts()

PlugType
Type 2 CCS        90
Type 2             9
Type 2 CHAdeMO     3
Type 1 CHAdeMO     1
Name: count, dtype: int64

In [16]:
encoder = LabelEncoder()
data['PlugType'] = encoder.fit_transform(data['PlugType'])

In [17]:
data.BodyStyle.value_counts()

BodyStyle
SUV          45
Hatchback    32
Sedan        10
Liftback      5
Pickup        3
Cabrio        3
SPV           3
MPV           1
Station       1
Name: count, dtype: int64

In [18]:
encoder = LabelEncoder()
data['BodyStyle'] = encoder.fit_transform(data['BodyStyle'])

In [19]:
data.Segment.value_counts()

Segment
C    30
B    22
D    15
F    12
E    10
A     7
N     6
S     1
Name: count, dtype: int64

In [20]:
encoder = LabelEncoder()
data['Segment'] = encoder.fit_transform(data['Segment'])

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    int64  
 1   Model            103 non-null    int64  
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    int64  
 7   RapidCharge      103 non-null    int64  
 8   PowerTrain       103 non-null    int64  
 9   PlugType         103 non-null    int64  
 10  BodyStyle        103 non-null    int64  
 11  Segment          103 non-null    int64  
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 11.4 KB


## Split the dataset for training and testing in ratio 80:20

In [22]:
from sklearn.model_selection import train_test_split

Y = data.PriceEuro
X = data.drop(['PriceEuro'], axis=1)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [67]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=42)

param_grid = {
    'max_depth': [3, 5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, Y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)

Best Parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 5}
Best Cross-Validation Score: -419682214.8107843


C:\Users\Nikola\anaconda3\envs\myenv\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
75 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
56 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Nikola\anaconda3\envs\myenv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Nikola\anaconda3\envs\myenv\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\Nikola\anaconda3\envs\myenv\Lib\site-packages\skl

In [68]:
from sklearn.metrics import r2_score

model = DecisionTreeRegressor(random_state=42, max_depth=10, max_features='sqrt', min_samples_split=2)

model.fit(X_train, Y_train)

DecisionTreeRegressor(max_depth=10, max_features='sqrt', random_state=42)

## Predict the outcomes for X test

In [69]:
tree_pred = model.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [70]:
r2_score(Y_test,tree_pred)

0.8106246185334829

## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [71]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

model = XGBRegressor(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],  # Number of boosting rounds
    'max_depth': [3, 6, 10],         # Maximum depth of each tree
    'learning_rate': [0.01, 0.1, 0.3] # Step size shrinking to prevent overfitting
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, Y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score}")

Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Best Cross-Validation Score: -421297202.3609562


In [72]:
model = XGBRegressor(max_depth=3, n_estimators=50, learning_rate=0.1, random_state=42)
model.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

## Predict the outcomes for X test

In [73]:
xgb_pred = model.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [74]:
r2_score(Y_test, xgb_pred)

0.7945261597633362

## Compare the performances of both model for at least three regression metircs

In [75]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

tree_mae = mean_absolute_error(Y_test, tree_pred)
xgb_mae = mean_absolute_error(Y_test, xgb_pred)

tree_mse = mean_squared_error(Y_test, tree_pred)
xgb_mse = mean_squared_error(Y_test, xgb_pred)

tree_r2 = r2_score(Y_test, tree_pred)
xgb_r2 = r2_score(Y_test, xgb_pred)

print("DecisionTreeRegressor Metrics:")
print(f"MAE: {tree_mae:.4f}")
print(f"MSE: {tree_mse:.4f}")
print(f"R²: {tree_r2:.4f}")

print("\nXGBRegressor Metrics:")
print(f"MAE: {xgb_mae:.4f}")
print(f"MSE: {xgb_mse:.4f}")
print(f"R²: {xgb_r2:.4f}")

DecisionTreeRegressor Metrics:
MAE: 7738.5873
MSE: 137437836.7778
R²: 0.8106

XGBRegressor Metrics:
MAE: 6721.3348
MSE: 149121189.5054
R²: 0.7945
